In [3]:
import pandas as pd
import glob
import datetime
import numpy as np

In [4]:
folder_file_pattern=glob.glob("/home/jian/Projects/Big_Lots/Sales_All/Sales_Data/*.txt")

In [5]:
taxonomy=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Product Taxonomy.txt",sep="|",dtype=str)
depart_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Department Names.txt",sep="|",dtype=str)
class_name=pd.read_csv("/home/jian/BigLots/static_files/MediaStorm Data Extract - Class Names.txt",sep="|",dtype=str,encoding = "ISO-8859-1")

In [6]:
depart_class=taxonomy[['department_id','class_code_id']].drop_duplicates()
depart_class=pd.merge(depart_class,depart_name,on='department_id',how="left")
depart_class=pd.merge(depart_class,class_name,on='class_code_id',how="left")

In [7]:
depart_class.head(2)

,department_id,class_code_id,department_desc,class_code_desc
0,108,11099,CANDYSNACK,FINANCE USE ONLY
1,111,11199,DIRECT STORE DELIVERIES,FINANCE USE ONLY


In [8]:
sales_all=pd.DataFrame()
for file in folder_file_pattern:
    df=pd.read_csv(file,sep="|",dtype=str,na_values="?")
    if "class_code_id" in df.columns:


        df['class_gross_sales_amt']=df['class_gross_sales_amt'].astype(float)
        df['class_gross_sales_amt']=df['class_gross_sales_amt'].fillna(0.0)
        df=df[['location_id','week_end_dt','class_code_id','class_gross_sales_amt']].drop_duplicates()
        df.reset_index(inplace=True)
        try:
            df['week_end_dt']=df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
            
        except:
            print(file,"Date Error")
            
            
    else:
        print (file,"Error: class_code_id not in columns")
        
    sales_all=sales_all.append(df)

In [17]:
# April + May
Q1_sale_2017=sales_all[(sales_all['week_end_dt']>=datetime.datetime(2017,4,2).date()) & (sales_all['week_end_dt']<=datetime.datetime(2017,6,6).date())]
Q1_sale_2018=sales_all[(sales_all['week_end_dt']>=datetime.datetime(2018,4,1).date()) & (sales_all['week_end_dt']<=datetime.datetime(2018,6,6).date())]

In [18]:
# April + May
Q1_sales_all=Q1_sale_2017.append(Q1_sale_2018)
Q1_sales_all=Q1_sales_all.drop_duplicates()
Q1_sales_all=pd.merge(Q1_sales_all,depart_class,on="class_code_id",how="left")
del Q1_sales_all['index']

In [19]:
Q1_sales_all["location_id"]=Q1_sales_all["location_id"].astype(int)

Q1_sales_all=Q1_sales_all.sort_values(["location_id","week_end_dt"])

In [29]:
Q1_sales_all=Q1_sales_all[(Q1_sales_all['location_id']!=145)&(Q1_sales_all['location_id']!=6990)]

In [30]:
Q1_sales_all.to_csv("/home/jian/Projects/Big_Lots/Sales_All/BL_Product by class April and May.csv",index=False)